# Chainerを用いた機械学習の基礎

## 機械学習とは

ニューラルネットワークを含む多くの機械学習における学習タスクは`最適なパラメータを探す問題`です。
最適なパラメータは目的関数の最小化（最大化）問題を解くことで自動的に得られます。

次のステップによって、自分の白州モデルを作っていくこととなります。

1. モデルの定義をする
2. 目的関数を定義する
3. 目的関数を最適化することで，モデルを学習させる。

## モデルの定義をする

はじめに学習対象のモデルを定義します。

ここでは，学習対象のモデルはいくつかのパラメータを使った関数だとします。
Pythonプログラムでいうと，学習対象のモデルはクラスのメソッドであり，パラメータはメンバ変数（インスタンス）のようなものです。

例えば次のクラスlayerはパラメータ $a$ と $b$ を持ち，関数としては $ax + b$を表します。<br>
この関数の挙動はパラメータ $a$ と $b$ を変えることで変わります。

```python
class layer(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, x):
        return self.a * x + self.b

l = layer(2.0, -1.0)
print(l(1.0)) # 1.0
print(l(2.0)) # 3.0
```


同様に，学習対象のモデルも複数のパラメータを持ち，それらパラメータを調整することで望むような挙動をするようにさせます。

### 層について

- 練習<br>

与える、引数を変えてみましょう。

In [13]:
class layer(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, x):
        return self.a * x + self.b


l = layer(2.0, -1.0)
print(l(5.0))  
print(l(8.0))  

9.0
15.0


Chainerでは学習可能なモデルをLinkとよびます。<br>

ディープラーニングで利用される代表的なLinkは `chainer.links` でサポートされています。<br>
また，自分で新しいLinkを作ることもできます。<br>
<br>
以降では，この `chainer.links` を `L` として使えるようにします。<br>
<br>

```python
from chainer import links as L
```
<br>
もっと基本的なLinkはLinearとよばれるLinkです。<br>
Linearは全ての入力と出力がつながっているようなニューラルネットワークを表します。<br>
Linearはニューラルネットワークの文脈では全結合層，数学の用語では線形変換，アフィン変換とよびます。<br>
たとえば，次の例では5個のユニットから，2個のユニットへの変換を表します。<br>
<br>
```python
lin = L.Linear(5, 2)
```
<br>
この `lin` はLinkオブジェクトですが，次のように関数呼び出しをすることができます。<br>
（この関数呼び出しは `__call__` で定義されており，演算子オーバーロードで実現されています。）<br>
<br>
```python
import numpy as np
from chainer import Variable

lin = L.Linear(5, 2)
x = Variable(np.ones((3, 5), dtype=np.float32))
y1 = lin(x)
```
<br>
この `numpy` ， `Variable` については後で詳しく説明します。<br>
この例である`np.ones((3, 5), dtype=np.float32)` は3行5列で全ての値が1であるような行列を作ります。<br>
`Variable` はその値に加えて学習に必要な情報が埋め込まれているオブジェクトとだけ覚えてください。<br>
つまりここでは3個の5次元のベクトルを用意し，それをVariableというオブジェクトにセットし，<br>
それを `lin` の引数として与えて，出力を `y` として計算しています。<br>
`lin`は5次元の入力を2次元の出力へ変換する関数なので，`y`は3個の2次元のベクトルになります。<br>

- 線形変換，アフィン変換 

線形変換（アフィン変換）は次のように表される変換です。

$$
\begin{align}
f(x; θ) &= Wx + b \\
θ &= (W, b)
\end{align}
$$

例えば上記例のLinearは，5次元のベクトルから2次元のベクトルへの線形変換を表します。

- 練習

上記の例を用いて，出力次元を4次元にして出力してください。

In [5]:
import numpy as np
from chainer import links as L
from chainer import Variable

link = L.Linear(5, 4)
x = Variable(np.ones((3, 5), dtype=np.float32))
y1 = link(x)

print(x)
print(y1)

variable([[1. 1. 1. 1. 1.]
          [1. 1. 1. 1. 1.]
          [1. 1. 1. 1. 1.]])
variable([[ 0.30907738 -0.9319836  -0.08234261  0.20045832]
          [ 0.30907738 -0.9319836  -0.08234261  0.20045832]
          [ 0.30907738 -0.9319836  -0.08234261  0.20045832]])


### 活性化関数について

Chainerでもう一つ重要なオブジェクトとしてFunctionがあります。<br>
FuncitonはLinkとは違って，学習可能なパラメータを持ちません。<br>
つまり，学習によって挙動を変えません。<br>
<br>
ディープラーニングで利用されている代表的な関数は `chainer.functions` で定義されています。<br>
また，自分で新しいFunctionを作ることもできます。<br>
<br>
以降では，この  `chainer.functions` をFとして使えるようにします。<br>
<br>
```python
from chainer import functions as F
```
<br>
例えば，ディープラーニングでよく使われるReLUとよばれる非線形関数 $f_{relu}$ は<br>
<br>
$$f_{relu}(x)=max(x,0)$$
<br>
で定義されます。<br>
つまり，もし $x$ が $0$ よりも大きければ $x$ をそのまま返し，もし小さければ $0$ を返すような関数です。<br>
<br>
Chainerでは次のように記述できます。<Br>
<br>
```python
from chainer import functions as F
...
y2 = F.relu(x)
```
<br>
これらのLinkとFunctionを組みわせて複雑な関数を作ることができます。<br>
例えば，前回の例のLinearを適用した後にReLUを適用した結果は次のように計算されます。<br>
<br>
```
y3 = F.relu(lin(x))
```

- 練習

先ほどのy1をReLU関数及び、もう一つの有名な関数`sigmoid関数`に与えてみましょう。<br>
<br>
・sigmoid
<br>
$$f_{sigmoid}(x)=\frac{1}{1+exp(-x)}$$
<br>
```
y3 = F.sigmoid(link(x))
```

In [6]:
from chainer import functions as F

y2 = F.relu(y1)
y3 = F.sigmoid(y1)

print(y2)
print(y3)

variable([[0.30907738 0.         0.         0.20045832]
          [0.30907738 0.         0.         0.20045832]
          [0.30907738 0.         0.         0.20045832]])
variable([[0.57666004 0.28252244 0.47942597 0.54994744]
          [0.57666004 0.28252244 0.47942597 0.54994744]
          [0.57666004 0.28252244 0.47942597 0.54994744]])


これまで扱ったLinkとFunctionを組み合わせて，学習対象のモデルを実際に作ってみましょう。<br>
<br>
以下に三層からなるニューラルネットワークの例をあげます。<br>
<br>
```python
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)
```
<br>
各機能は今後詳細に説明されますので，ここでは概要だけ説明します。<br>
詳細は理解できなくてもそのまま飛ばして問題ありません。<br>
<br>
このMLPは，三つのLinear（l1, l2, l3）を学習可能なパラメータとして持ち，<br>
`__call__`でそれらのパラメータを利用して結果を計算します。<br>
<br>
なお，`L.Linear` の第一引数には `None` を指定することで実際の入力からユニット数を自動で設定してくれます。<br>
<br>
`__call__` では先ほど定義した層に入力`x`を与えて計算（順計算）を行います。<br>
まず `l1` に大元の入力 `x` を与え，それをLinearで変換したものにReLUを適用します。<br>
その計算結果 `h1` を次の層 `l2` に与え同様の計算を行います。<br>
`l3` に関しても同様に前層の結果を元に計算を行います。<br>
最終的に `l3` の結果を返すことで計算が完了します。<br>
<br>
このように<br>
(1)Linkを使って学習対象のパラメータを定義<br>
(2)次にそれらを使って順計算を定義<br>
することで学習対象のモデルを定義できます。

In [11]:
import chainer
from chainer import links as L
from chainer import functions as F
from chainer import Chain

from chainer import Variable

class MLP(Chain):
    def __init__(self, n_unit, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_unit)
            self.l2 = L.Linear(None, n_unit)
            self.l3 = L.Linear(None, n_out)
            
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        
        return y

In [12]:
model = MLP(5, 2)
x = Variable(np.ones((4, 5), dtype=np.float32))
y = model(x)

## 目的関数の定義

目的関数は一つの値を出力し，値が小さければ望ましい状態を表すような関数です。<br>
<br>
例えば，訓練データに対し学習対象モデルが予測をし，間違えた回数を $L$ とします。<br>

この間違えた回数 $L$ を小さくするということは，学習対象モデルが訓練データをたくさん当てられるようにすることを意味します。<br>

この場合，目的関数は学習対象モデルを引数として受取り，間違えた数を返すような関数です。<br>

### 出力層における関数

今回，学習したいモデルは入力 $x$ が与えられた時に、<br>
出力 $y$ の条件付き確率 $p(y \mid x)$ を出力してくれるようなモデルです。<br>
<br>
例えば，入力$x$に犬が写っていれば $p(犬 \mid x) = 0.99, p(猫 \mid x) = 0.01$ となるようなモデルです。
<br>
このようなカテゴリ値（離散値）に対する確率分布をモデル化するにはSoftmaxを利用します。<br>
<br>
Chainerでは `chainer.functions` で `softmax` 関数が定義されているのでそれを使いましょう。<br>
<br>
例えば，入力`x`を何らかのモデルで変換しカテゴリ種類数と同じ次元数を持つベクトル`t`を作ります。<br>
次に（必ずしも確率分布となっていない）ベクトル`t`を`softmax`を使って確率分布に変換します。<br>
<br>
```python
t = model(x)
y = F.softmax(t)
```
<br>
- Softmax
<br>
Softmax（または多クラスロジスティックスモデル）とは $d$ 次元の実数値ベクトルから，<br>
$d$ 次元の確率分布を作る方法の一つです。<br>
<br>
softmaxは，$x$ が $d$ 次元であり，各次元の値が $x[0], x[1], ..., x[d-1]$ の時，<br>
<br>
$$y[i]=\frac{\exp(x[i])}{\sum_j \exp(x[j])}$$
<br>
と表されます。<br>
<br>

あるベクトルvが確率分布となる条件として，<br>

1. 各次元の値が非負
2. 合計値が1

という条件があります。<br>
Softmaxは， $\exp$ が非負であることから1の条件を満たし，<br>
各次元の値を足した値で割っていることから2の条件を満たします。<br>

### 目的関数

- クロスエントロピー損失関数

学習の目標は学習データ$D$の確率分布 $p(y|x)$ と，
学習対象のモデルによる確率分布 $q(y \mid x; \theta)$ が一致するようにすることです。<br>
確率分布間がどれだけ離れているかを表す指標としてKLダイバージェンスが知られています。<br>
KLダイバージェンスは二つの確率分布 $P$ と $Q$ の遠さを次のように定義します。<br>
<br>
$$
\begin{align}
KL(P \mid\mid Q) &= \sum_x P(x) \log \frac{P(x)}{Q(x)} \\
                 &= \sum_x P(x) \log P(x) - \sum_x P(x) \log Q(x)
\end{align}
$$
<br>
もし， $P$ と $Q$ が同じならば，全ての $x$ について $\frac{P(x)}{Q(x)}=1$ となるので 
$KL(P \mid\mid Q)=0$ となります。<br>
この二つの分布が違うと， $KL(P \mid\mid Q)>0$ となり，近ければ近いほど0に近づくような指標です。<br>
<br>
学習データによって定義される確率分布は訓練分布と呼ばれ，それに基づく条件付き確率は<br>
<br>
$$
\begin{align}
P(y \mid x) &= \frac{P(x, y)}{P(x)} \\
            &= \frac{\sum_{i}I(x=x_i, y=y_i)}{\sum_{i}I(x=x_i)}
\end{align}
$$
<br>
と表されます。<br>
但し， $I$ はデルタ関数とよばれ， $I(c)$ は $c$ が真である時は $1$，それ以外は $0$ であるような関数です。<br>
<br>

訓練分布に基づく条件づき確率 $P(y \mid x)$ と，モデルによる条件づき確率 $Q(y \mid x)$ のKLダイバージェンスは<br>
<br>
$$
\begin{align}
KL(P \mid\mid Q) &= \sum_{x} \sum_y P(y \mid x) \log \frac{P(y \mid x)}{Q(y \mid x)} \\
                 &= \sum_{x, y} P(y \mid x) \log P(y \mid x) - \sum_{x, y} P(y \mid x) \log Q(y \mid x)
\end{align}
$$
<br>
となります。<br>
この最適化において， $Q$ に依存する項は第二項のみであり， $(x, y) \in D$ の時 $P(y \mid x)=1$ ，それ以外 $0$ ですので<br>
<br>
$L(\theta) = - \sum_{i=1}^n \log Q(y_i \mid x_i)$
<br>
となります。<br>
これをクロスエントロピー損失関数，または負の対数尤度ともよばれます。<br>
<br>
ここまで読んだ方で，なぜ学習データ $D$ から得られた確率分布 $P$ そのものを直接使わず，
学習モデルによる確率 $Q$ を使うのかと思った方がいるかもしれません。<br>
それは，学習の目標は学習データだけをうまく分類することではなく未知のデータをうまく分類することだからです。<br>
<br>
$P$ は，入力が学習データと全く同じであればそれが正解となりますが，そうでない場合は確率分布が不定になりえます。<br>
$Q$ はありえそうなモデルの中で一番 $P$ に近い分布を探しています。<br>
$Q$ は $P$ とは違って全ての $x$ について確率分布を与えることができるため，<br>
学習データには含まれない未知のデータでもうまく分類できます。<br>
別の言い方をすると$P$を平滑化した確率分布が$Q$となります。<br>

学習を行うために，入力と正解の出力のペアからなる $n$ 個の学習データ
<br>
$$D = \{(x_1, y_1), (x_2, y_2), ..., (x_n, y_n)\}$$
<br>
を用意します。<br>
<br>
この学習データと，学習対象のモデルが一致するように，つまり学習データそれぞれ $(x_i, y_i)$ に対し， <br>
$p(y_i \mid x_i)$ が大きい時に，値が小さくなるような目的関数を用意します。<br>
<br>
ここでは代表的な関数である`softmax_cross_entropy`とよばれる関数を使います。<br>
<br>
```python
## x は入力, tは正解の出力
h = MLP(x)
loss = F.softmax_cross_entropy(h, t)
```
<br>
ChainerではSoftmaxを適用した後に，クロスエントロピー損失関数を適用した目的関数が用意されています。<br>
これは，二つまとめて処理をしないと，数値誤差の問題があるためです。<br>